In [88]:
import nltk
from collections import Counter
import pandas as pd
import time

import csv
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize,blankline_tokenize
from nltk import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
import re
import string
from collections import Counter
import json
import re as regex

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.model_selection import StratifiedKFold



In [89]:
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    featureList = []
    fea_vect=[]
    whitelist = ["n't","not","don't","up","down","below"]    
    
    data_model = None
    data_labels = None
    is_testing = False
    
    def initialize(self, csv_file, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            #self.data_model = pd.read_csv(from_cached)
            self.data_model = pd.read_json(from_cached)

            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            #self.data = pd.read_csv(csv_file, header=0, names=["id", "emotion", "text"])
            self.data = pd.read_json(csv_file)

            #self.data = self.data[self.data["emotion"].isin(["positive", "negative", "neutral"])]
        

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None
        
    
    
    
    def do_process(self):
        import time
        start_time = time.time()
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                #remove numbers
                def remove_numbers(text):
                    return re.sub(r'\d+', '', text)

                #20 Function to remove whitespace
                def remove_whitespace(text):
                    return " ".join(text.split())

                # Function to remove punctuations
                def remove_punctuations(text):
                    words = nltk.word_tokenize(text)
                    punt_removed = [w for w in words if w.lower() not in string.punctuation]
                    return " ".join(punt_removed)
#30

                # Function to remove stop words
                def remove_stopwords(text, lang='english'):
                    whitelist = ["n't","not","don't","up","down","below"]    

                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    #filtered_sentence = [w for w in word_tokens if not w in stop_words]
                    filtered_sentence = []
                    for w in word_tokens:
                        if w not in stop_words or w in whitelist:
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40




                # Function to extract n-grams from text
                def get_ngrams(text, n):

                    n_grams = ngrams(nltk.word_tokenize(text), n)
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams


                # Function to apply lemmatization to a list of words
                def words_lemmatizer(text, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    words = nltk.word_tokenize(text)
                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return " ".join(lemma_words)

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
            
                idx=row["spans"]
                #convert to string
                #ch=""
                #for w in idx :
                ch=" ".join(idx)
    
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                txt_num=remove_numbers(ch)
    
                txt_ws=remove_whitespace(txt_num)
    
                txt_pun=remove_punctuations(txt_ws)
        
                txt_final=remove_stopwords(txt_pun)

                texte=words_lemmatizer(txt_final)

                n_grams=get_ngrams(texte, 1)
                row["token_spans"] = n_grams
    
                return row
        self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        print("--- %s seconds ---" % (time.time() - start_time))

    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                       whitelist=None):
        self.wordlist = []
        whitelist = self.whitelist if whitelist is None else whitelist
        

        words = Counter()

        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        
                

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_copie.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        label_column = []
        
        label_column = ["label"]

        columns = label_column + list(
            map(lambda w: w + "_bow",self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if True:
                # add label
                current_label = self.processed_data.loc[idx, "sentiment score"]
                labels.append(current_label)
                current_row.append(current_label)

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels


In [90]:
class TwitterData_Initialize_test(TwitterData_Initialize):

    
    
    def do_process(self):
        def stem_and_join(row,stemmer=nltk.PorterStemmer()):
            row["spans"] = list(map(lambda str: stemmer.stem(str.lower()), row["spans"]))
            return row
    
        def tokenize_grams(row):
                #remove numbers
                def remove_numbers(text):
                    return re.sub(r'\d+', '', text)

                def to_lower(text):
                    return text.lower()
                
                #20 Function to remove whitespace
                def remove_whitespace(text):
                    return " ".join(text.split())

                # Function to remove punctuations
                def remove_punctuations(text):
                    text=re.sub(r'[?|*|.|!|+|-]',r'',text)

                    words = nltk.word_tokenize(text)
                    punt_removed = [w for w in words if w.lower() not in string.punctuation]
                    return " ".join(punt_removed)
#30

                # Function to remove stop words
                def remove_stopwords(text, lang='english'):
                    whitelist = ["n't","not","below"]    

                    stop_words = set(stopwords.words('english'))
                    word_tokens = word_tokenize(text)
                    #filtered_sentence = [w for w in word_tokens if ((not w in stop_words) or (w in whitelist))]
                    filtered_sentence = []
                    for w in word_tokens:
                        if ((w not in stop_words) or (w in whitelist)):
                            filtered_sentence.append(w)
                
                    ch=" ".join(filtered_sentence)

                    return ch
#40




                # Function to extract n-grams from text
                def get_ngrams(text, n):

                    n_grams = ngrams(nltk.word_tokenize(text), n)
                    list_grams=[ ' '.join(grams) for grams in n_grams]
                    return list_grams


                # Function to apply lemmatization to a list of words
                def words_lemmatizer(text, encoding="utf8"):
                    wordnet_lemmatizer = WordNetLemmatizer()

                    words = nltk.word_tokenize(text)
                    lemma_words = []
                    wl = WordNetLemmatizer()
                    for word in words:
                        pos = find_pos(word)
                        lemma_words.append(wl.lemmatize(word, pos))
                    return " ".join(lemma_words)

                # Function to find part of speech tag for a word
                def find_pos(word):
                    # Part of Speech constants
                    # ADJ, ADJ_SAT, ADV, NOUN, VERB = 'a', 's', 'r', 'n', 'v'
   
                    pos = nltk.pos_tag(nltk.word_tokenize(word))[0][1]
                    # Adjective tags - 'JJ', 'JJR', 'JJS'
                    if pos.lower()[0] == 'j':
                        return 'a'
                    # Adverb tags - 'RB', 'RBR', 'RBS'
                    elif pos.lower()[0] == 'r':
                        return 'r'
                    # Verb tags - 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'
                    elif pos.lower()[0] == 'v':
                        return 'v'
                    # Noun tags - 'NN', 'NNS', 'NNP', 'NNPS'
                    else:
                        return 'n'
    
            
            
                idx=row["spans"]
                ch=''.join(idx)


    
                #words_stemmer(ch, type="PorterStemmer", lang="english", encoding="utf8")
                #Convert to lower case
                txt_num=remove_numbers(ch)
                
                txt_low=to_lower(txt_num)
                
                txt_ws=remove_whitespace(txt_low)
    
                txt_pun=remove_punctuations(txt_ws)
        
                txt_final=remove_stopwords(txt_pun)

                texte=words_lemmatizer(txt_final)

                n_grams=get_ngrams(texte, 1)
                print("list ngrams--------------\n",n_grams)

                row["token_spans"] = n_grams
    
                return row
        #self.processed_data = self.processed_data.apply(stem_and_join, axis=1)
        self.processed_data = self.processed_data.apply(tokenize_grams, axis=1)
        
    def build_wordlist(self, min_occurrences=0, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                      ):
        self.wordlist = []
        

        words = Counter()

        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "token_spans"])

        
                

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist_test.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]
        
        
    def build_data_model(self):
        

        columns = list(
            map(lambda w: w + "_bow",self.wordlist))
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "token_spans"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        return self.data_model


In [91]:
data = TwitterData_Initialize()
data.initialize("Microblog_Trialdata.json")

data.processed_data.head()

,cashtag,id,sentiment score,source,spans
0,$F,5540055,-0.454,stocktwits,[Putting on a little $F short]
1,$AAPL,10752226,-0.464,stocktwits,[short some]
2,$BAC,10920221,0.445,stocktwits,[buying opportunity]
3,$SHOR,12971398,0.661,stocktwits,[Scaling Up on Long Position]
4,$JPM,16142438,-0.763,stocktwits,[its time to sell banks]


In [92]:
data_test = TwitterData_Initialize_test()
data_test.initialize("Microblog_trialtest.json")

data_test.processed_data.head()


,cashtag,id,source,spans
0,$SBUX,719911771625140200,twitter,will be a solid long term investmen Income
1,$AAPL,711948847971307500,twitter,Excited;$;big +value;$;Hoping little
2,$QUNR,719917003033681900,twitter,China's Tourist Bonanza Isn't Going Anywhere
3,$ESI,39047884,stocktwits,0.15
4,$TSLA,719590251891658700,twitter,not guaranteed


In [79]:
from scipy import sparse
start_time = time.time()
tweets=list(data.processed_data.spans)
tweets
list_tweet=[]
for i,chaine in enumerate(tweets):
    ch="".join(chaine)
    list_tweet.append(ch)
list_tweet
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(list_tweet)
print("Vocabulary size: {}".format(len(vect.vocabulary_)))
print("Vocabulary content:\n {}".format(vect.vocabulary_))
X = vect.transform(list_tweet)
print("bag_of_words: {}".format(repr(X)))
print("Dense representation of bag_of_words:\n{}".format(
X.toarray()))
print("--- %s seconds ---" % (time.time() - start_time))

#sparse_matrix = sparse.csr_matrix(bag_of_words)
#print("\nSciPy sparse CSR matrix:\n{}".format(sparse_matrix))


Vocabulary size: 57
Vocabulary content:
 {'putting': 36, 'on': 29, 'little': 21, 'short': 41, 'some': 43, 'buying': 8, 'opportunity': 30, 'scaling': 39, 'up': 51, 'long': 22, 'position': 33, 'its': 20, 'time': 49, 'to': 50, 'sell': 40, 'banks': 4, 'entering': 13, 'picked': 32, 'accumulate': 2, 'for': 14, 'positionfar': 34, 'more': 26, 'upside': 52, 'than': 47, 'downside': 12, 'looking': 23, 'strong': 44, 'bouncelunchtime': 7, 'rally': 37, 'coming': 9, 'very': 53, 'intrigued': 18, 'with': 54, 'the': 48, 'technology': 46, 'and': 3, 'growth': 17, 'potential': 35, 'short2': 42, '48': 0, 'workedputs': 55, 'biggest': 6, 'market': 25, 'losers': 24, 'goog': 15, 'googl': 16, 'would': 56, 'suck': 45, 'sbux': 38, 'dip': 11, 'is': 19, 'below': 5, '740and': 1, 'overbought': 31, 'not': 28, 'non': 27, 'didn': 10}
bag_of_words: <16x57 sparse matrix of type '<class 'numpy.int64'>'
	with 74 stored elements in Compressed Sparse Row format>
Dense representation of bag_of_words:
[[0 0 0 0 0 0 0 0 0 0 0 0 0

In [93]:
data.processed_data.head(20)

,cashtag,id,sentiment score,source,spans
0,$F,5540055,-0.454,stocktwits,[Putting on a little $F short]
1,$AAPL,10752226,-0.464,stocktwits,[short some]
2,$BAC,10920221,0.445,stocktwits,[buying opportunity]
3,$SHOR,12971398,0.661,stocktwits,[Scaling Up on Long Position]
4,$JPM,16142438,-0.763,stocktwits,[its time to sell banks]
5,$LMT,14073133,0.627,stocktwits,[Entering long]
6,$DNN,18479024,0.653,stocktwits,[picked some up]
7,$CRK,34147106,0.668,stocktwits,"[time to accumulate for a long position, far m..."
8,$CRK,34147106,0.460,stocktwits,"[Looking for a strong bounce, Lunchtime rally ..."
9,$CRK,34147106,0.403,stocktwits,[Very intrigued with the technology and growth...


In [94]:
data.do_process()
data.processed_data.head(20)
data.build_wordlist()
data.wordlist
bow, labels = data.build_data_model()
len(bow.columns)


--- 2.251349449157715 seconds ---


46

In [95]:
Y=data.processed_data["sentiment score"].values
Y = bow.label.as_matrix()
X_drop = bow.drop(["label"],axis=1)
#train_x = train_x.drop(["ID"],axis=1)
#train_x = train_x.values
X=X_drop.as_matrix()

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,
                                                    train_size=0.7
                                                    )


In [102]:
# grid search
model = XGBRegressor()
subsample = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
param_grid = dict(subsample=subsample)
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1)
grid_result = grid_search.fit(X_train, y_train)
# summarize results


JoblibAttributeError: JoblibAttributeError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\Globalnet\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\Globalnet\Anacon

In [10]:

#Fitting XGB regressor 
model = xgb.XGBRegressor()
model.fit(X_train,y_train)
print (model)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)


In [11]:
#Predict 
output = model.predict(X_test)
final_df = pd.DataFrame()
#final_df["ID"] = id_vals
#final_df["cashtag"]=row2

#final_df["spans"]=row
final_df["Prediction"] = output
#final_df.to_csv("Output_1.csv",sep=",")
final_df.head(20)

,Prediction
0,0.132158
1,0.153285
2,0.179562
3,-0.219891
4,0.018351
5,0.133477
6,0.133477
7,0.133477
8,0.133477
9,0.116063


In [12]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
acc=mean_squared_error(output, y_test)

print("Accuracy:" ,acc)

Accuracy: 0.10838428827316153


--- 2.50732421875 seconds ---


,cashtag,id,sentiment score,source,spans,token_spans
0,$F,5540055,-0.454,stocktwits,[putting on a little $f short],"[put, little, f, short]"
1,$AAPL,10752226,-0.464,stocktwits,[short som],"[short, som]"
2,$BAC,10920221,0.445,stocktwits,[buying opportun],"[buying, opportun]"
3,$SHOR,12971398,0.661,stocktwits,[scaling up on long posit],"[scale, up, long, posit]"
4,$JPM,16142438,-0.763,stocktwits,[its time to sell bank],"[time, sell, bank]"
5,$LMT,14073133,0.627,stocktwits,[entering long],"[enter, long]"
6,$DNN,18479024,0.653,stocktwits,[picked some up],"[picked, up]"
7,$CRK,34147106,0.668,stocktwits,"[time to accumulate for a long posit, far more...","[time, accumulate, long, posit, far, upside, d..."
8,$CRK,34147106,0.460,stocktwits,"[looking for a strong bounc, lunchtime rally com]","[look, strong, bounc, lunchtime, rally, com]"
9,$CRK,34147106,0.403,stocktwits,[very intrigued with the technology and growth...,"[intrigue, technology, growth, potenti]"


['short',
 'put',
 'up',
 'long',
 'little',
 'f',
 'buying',
 'posit',
 'time',
 'not',
 'som',
 'opportun',
 'scale',
 'sell',
 'bank',
 'enter',
 'picked',
 'accumulate',
 'far',
 'upside',
 'downsid',
 'look',
 'strong',
 'bounc',
 'lunchtime',
 'rally',
 'com',
 'intrigue',
 'technology',
 'growth',
 'potenti',
 '***work',
 'big',
 'market',
 'los',
 'goog',
 'googl',
 'would',
 'suck',
 'sbux',
 'dip',
 'below',
 'overbought',
 'non',
 "n't"]

1770

In [20]:
time.clock()

42.14956432827035

In [13]:
stop_words = set(stopwords.words('english'))

In [14]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r